In [1]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# read ROIs from geojson file

In [2]:
import json

roiFile=r".\files\Multi_Class_Land_Cover_Change_AOIs.geojson"

with open(roiFile) as f:
    data = json.load(f)
    
for feature in data['features']:
    #print(feature['properties'])
    #featureGEE=ee.Feature(geojsonDict)
    
    '''convert to gee feature'''
    #jsonDict = eval(geom.to_json())
    #geojsonDict = jsonDict['features'][0]

    cor=feature['geometry']['coordinates']
         
    aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
    #print(aoi)

In [ ]:
Map = geemap.Map()
Map

# visualize one aoi

In [ ]:
feature =data['features'][0]

cor=feature['geometry']['coordinates']     
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)

print(feature['properties']['title'])
print(feature['properties'])

print(cor[0][0][0], cor[0][0][1])


In [ ]:
def get_index(lst=None, item=''):
         return [i for i in range(len(lst)) if lst[i]['properties']['title'] == item]

get_index(data['features'], "8079_4130_13")

In [ ]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,13)
Map.addLayer(aoi, {'color': 'FF0000'})

# download monthly s2 data for one aoi 

## setting parameters

In [ ]:
timePeriod = ee.DateRange('2018-01-01', '2018-02-01')
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
#['Blue', 'Green', 'Red', 'Red Edge 1', 'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4', 'SWIR1', 'SWIR2']

import os
saveFolder= r".\files\res"
filename = os.path.join(saveFolder, feature['properties']['title']+'_201801'+'.tif')

## get utm_epsg from (lon, lat)

In [3]:
def get_utm_epsg(lon, lat):
	#coords = feature.geometry().coordinates()
	lon = ee.Number(lon)
	lat = ee.Number(lat)
	epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
	#print(epsg.getInfo())
	return ee.String("EPSG:").cat(ee.String(str(epsg.getInfo())))#ee.String("EPSG:").cat

In [ ]:
#ee.ImageCollection("COPERNICUS/S2_SR")
#https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

def getS2(aoi, start_date, end_date):
	# Define function to mask out clouds and cloud shadows in images.
	# Use CFmask band included in USGS Landsat SR image product.
	def fmask(img):
		cloudOpaqueBitMask = 1 << 10
		cloudCirrusBitMask = 1 << 11
		qa = img.select('QA60')
		mask = qa.bitwiseAnd(cloudOpaqueBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
		return(img.updateMask(mask))
               
	image = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(start_date, end_date) \
	.filterBounds(aoi).map(fmask).select(s2_bands).median()

	return image.clip(aoi)


In [ ]:
#s2 = getS2(aoi, timePeriod)
s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(timePeriod).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)

#print(s2.getInfo())

'for debug'
#print(s2.getInfo())
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2")

In [ ]:
epsg=get_utm_epsg(cor[0][0][0], cor[0][0][1])
print(epsg.getInfo())

In [ ]:
print(filename[:-4]+".zip")

In [ ]:
geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
os.remove(filename[:-4]+".zip")

# loop months for downloading

In [ ]:
year=2018

for m in range(1,12):
    
    start_date = ee.Date.fromYMD(year, m, 1)
    end_date = ee.Date.fromYMD(year, m+1, 1)
    
    s2C =  ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(start_date, end_date).select(s2_bands) \
	.filterBounds(aoi)
    
    print(s2C.size().getInfo())
    
    s2 = s2C.median().clip(aoi)
    
    filename = os.path.join(saveFolder, feature['properties']['title']+'_'+str(year)+'_'+str(m)+'.tif')
    
    geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
    
    os.remove(filename[:-4]+".zip")

# find a less cloudy image, how?

In [ ]:

year=2018

for m in [1,2,3,10,11]:#range(1,6):
    
    start_date = ee.Date.fromYMD(year, m, 1)
    end_date = ee.Date.fromYMD(year, m+1, 1)
    
    s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(start_date, end_date).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)
    
    s2_masked = getS2(aoi,start_date, end_date)
    
    # Convert image composite list to collection
    # imgCol = ee.ImageCollection.fromImages([s2,s2_masked,s2_masked])
    # s2_pro = imgCol.median()
    
    
    Map.addLayer(s2_masked.firstNonZero(s2), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, str(m))
    Map.addLayer(s2_masked, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "s2_masked"+str(m), False)
    Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "s2"+str(m), False)
    
    #filename = os.path.join(saveFolder, feature['properties']['title']+'_'+str(year)+'_'+str(m)+'.tif')
    
    #geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
    
    #os.remove(filename[:-4]+".zip")


# download all available s2 data

In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [10]:
roiFile=r".\files\Area_150_2.geojson"

with open(roiFile) as f:
    data = json.load(f)

#print(data)

roi =  data['features'][0]
#print(roi)

cor = roi['geometry']['coordinates']
#print(cor)
aoi = ee.Geometry.MultiPolygon(cor, None, False)


#aoi = aoi.buffer(100).bounds().simplify(ee.ErrorMargin(1, "meters"))
Map.centerObject(aoi,10)
Map.addLayer(aoi, {'color': 'FF0000'})

epsg = get_utm_epsg(cor[0][0][0][0], cor[0][0][0][1])

start_date = ee.Date.fromYMD(2018, 1, 1)

end_date = ee.Date.fromYMD(2018, 2, 1)

s2_all = ee.ImageCollection('COPERNICUS/S2').filterDate(start_date, end_date).filterBounds(aoi)#.clip

print("the available s2 data for this roi and time: ", s2_all.size().getInfo())
#print(s2_all.get('id').getInfo())
Map.addLayer(s2_all.median().clip(aoi), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000})

s2_all_ = s2_all.toList(s2_all.size().getInfo())
for i in range(s2_all.size().getInfo()):
    print(ee.Image(s2_all_.get(i)).get('PRODUCT_ID').getInfo())
    #Here the first numeric part represents the sensing date and time, the second numeric part represents 
    #the product generation date and time, 
    #and the final 6-character string is a unique granule identifier indicating its UTM grid reference

    #Map.addLayer(ee.Image(s2_all_.get(i)).clip(aoi), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, str(i))

the available s2 data for this roi and time:  12
None
S2A_MSIL1C_20180102T102421_N0206_R065_T33UVU_20180102T123237
S2B_MSIL1C_20180104T101409_N0206_R022_T33UVU_20180104T121410
S2B_MSIL1C_20180107T102359_N0206_R065_T33UVU_20180107T121759
S2A_MSIL1C_20180109T101401_N0206_R022_T33UVU_20180117T202124
S2A_MSIL1C_20180112T102401_N0206_R065_T33UVU_20180112T122623
S2B_MSIL1C_20180114T101349_N0206_R022_T33UVU_20180114T121005
S2B_MSIL1C_20180117T102339_N0206_R065_T33UVU_20180117T122826
S2A_MSIL1C_20180119T101331_N0206_R022_T33UVU_20180119T135441
S2A_MSIL1C_20180122T102321_N0206_R065_T33UVU_20180122T140448
S2B_MSIL1C_20180124T101309_N0206_R022_T33UVU_20180124T190119
S2B_MSIL1C_20180127T102259_N0206_R065_T33UVU_20180127T122959
S2A_MSIL1C_20180129T101251_N0206_R022_T33UVU_20180129T135502
